In this notebook we make models for clusters with unimodal fire season using the delta time series.

We use the functions defined in the file "scripts/modelling_functions_new.R".

In [1]:
setwd("..")

In [57]:
source("scripts/modelling_functions_new.R")

In [58]:
# Data required
library('ggplot2')
load('fireSeason_def_2.Rdata', verbose = T)
load("data/ba_mon_time_series_masked.Rdata", verbose = T)
load("data/ba_mon_clim_masked_df.Rdata", verbose = T)
load('corrDfDeltaAnnualClusV2.Rdata', verbose = T)
load('corrDfDeltaAnnualClusLag1V2.Rdata', verbose = T)
load('corrDfDeltaAnnualClusLag2V2.Rdata', verbose = T)
load('corrDfDeltaAnnualClusLag3V2.Rdata', verbose = T)

Loading objects:
  fireSeason_def_2
Loading objects:
  masked_coords
  dates
  masked_ba_series
Loading objects:
  df_masked
  masked_coords
Loading objects:
  corr.df.delta
Loading objects:
  corr.df.delta.1
Loading objects:
  corr.df.delta.2
Loading objects:
  corr.df.delta.3


## Climate indexes data loading

In [59]:
nina34 = read.table("CPC/nina34.anom.data.txt", skip = 3, nrows = 72)
colnames(nina34) = c("Year", 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec')
nina34[,-1] = (nina34[,-1] - mean(as.matrix(nina34[32:61,-1]))) / sd(as.matrix(nina34[32:61,-1]))

In [60]:
nao = read.table("CPC/nao_index.tim.txt", skip = 8, header = T)
nao <- nao %>% spread(MONTH,INDEX)
colnames(nao) = c("Year", 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec')

In [61]:
poleur = read.table("CPC/poleur_index.tim", skip = 8, header = T)
poleur <- poleur %>% spread(MONTH,INDEX)
colnames(poleur) = c("Year", 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec')

In [62]:
ea = read.table("CPC/ea_index.tim.txt", skip = 8, header = T)
ea <- ea %>% spread(MONTH,INDEX)
colnames(ea) = c("Year", 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec')

In [63]:
eawr = read.table("CPC/eawr_index.tim.txt", skip = 8, header = T)
eawr <- eawr %>% spread(MONTH,INDEX)
colnames(eawr) = c("Year", 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec')

In [64]:
pna = read.table("CPC/pna_index.tim.txt", skip = 8, header = T)
pna <- pna %>% spread(MONTH,INDEX)
colnames(pna) = c("Year", 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec')

In [65]:
scand = read.table("CPC/scand_index.tim.txt", skip = 8, header = T)
scand <- scand %>% spread(MONTH,INDEX)
colnames(scand) = c("Year", 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec')

In [66]:
wp = read.table("CPC/wp_index.tim.txt", skip = 8, header = T)
wp <- wp %>% spread(MONTH,INDEX)
colnames(wp) = c("Year", 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec')

# Models with no persistence

In [67]:
lm.list <- lm.all(fireSeason_def_2, log1p(masked_ba_series), dates, corr.df.delta, 
                  list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', useDeltas = T)
lm.list$results

,biome,cluster,lm.Npred,lm.RMSE,lm.bias,lm.RVar,lm.cor.pvalue,lm.cor,lm.acc,lm.acc.t1,lm.acc.t2,lm.acc.t3
2,1,1,3,644.1613,0.22546322,0.6091194,0.024138453,0.5285243,0.4444444,0.5000000,0.3333333,0.5000000
3,1,2,2,814.5127,0.22037978,0.5546609,0.002693328,0.6632875,0.5000000,0.5000000,0.3333333,0.6666667
4,1,3,1,1198.8092,0.63439353,0.3739380,0.072363808,0.4334172,0.5000000,0.5000000,0.3333333,0.6666667
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,2,1342.5594,0.67584986,0.4748432,0.014466709,0.5654238,0.5000000,0.6666667,0.3333333,0.5000000
7,1,6,1,1167.0856,0.32073172,0.2224139,0.184705134,0.3274423,0.5000000,0.5000000,0.5000000,0.5000000
8,2,1,0,NA,NA,NA,NA,NA,NA,NA,NA,NA
9,2,2,0,NA,NA,NA,NA,NA,NA,NA,NA,NA
10,2,3,2,1645.8899,6.34254428,0.6090327,0.002455416,0.6678372,0.6111111,0.6666667,0.6666667,0.5000000
11,2,4,0,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [68]:
rf.list <- rf.all(fireSeason_def_2, log1p(masked_ba_series), dates, corr.df.delta, 
                  list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', useDeltas = T)
rf.list$results

,biome,cluster,rf.mtry,rf.Ntree,rf.maxnodes,rf.RMSE,rf.bias,rf.RVar,rf.cor.pvalue,rf.cor,rf.acc,rf.acc.t1,rf.acc.t2,rf.acc.t3
2,1,1,3,100,7,12.043180,0.0036252446,0.12755036,0.394390660,-0.2073285360,0.4210526,0.3333333,0.3333333,0.5714286
3,1,2,1,100,11,20.692037,0.0074938632,0.04438985,0.308751031,-0.2466204301,0.4210526,0.3333333,0.5000000,0.4285714
4,1,3,2,175,9,27.336532,0.0163494164,0.08808394,0.471135505,-0.1759741747,0.4210526,0.3333333,0.6666667,0.2857143
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,2,75,5,12.581010,0.0038363671,0.07846591,0.668623277,-0.1050584889,0.3157895,0.1666667,0.5000000,0.2857143
7,1,6,1,10,5,17.915885,0.0044525639,0.14802975,0.026814446,-0.5067446490,0.2105263,0.0000000,0.3333333,0.2857143
8,2,1,8,125,7,15.768969,0.0362589797,0.28730119,0.306848156,0.2551552953,0.4444444,0.5000000,0.3333333,0.5000000
9,2,2,7,75,11,18.505578,0.0132972481,0.12742591,0.043454631,-0.4557138580,0.3000000,0.2857143,0.5000000,0.1428571
10,2,3,8,10,11,9.199470,0.0151718063,0.25981591,0.320030964,0.2411094099,0.5789474,0.5000000,0.6666667,0.5714286
11,2,4,2,25,7,11.029273,0.0116727580,0.12926893,0.137577123,0.3535408726,0.5789474,0.5000000,0.5000000,0.7142857


In [69]:
knn.list <- knn.all(fireSeason_def_2, log1p(masked_ba_series), dates, corr.df.delta, 
                  list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', useDeltas = T)
knn.list$results

,biome,cluster,k,knn.RMSE,knn.bias,knn.RVar,knn.cor.pvalue,knn.cor,knn.acc,knn.acc.t1,knn.acc.t2,knn.acc.t3
2,1,1,3,14.878849,0.0035797410,0.28702830,0.006411374,-0.601783979,0.2105263,0.1666667,0.3333333,0.1428571
3,1,2,3,18.865581,0.0075417527,0.24112970,0.224008423,0.292665424,0.3157895,0.3333333,0.1666667,0.4285714
4,1,3,4,25.877685,0.0162244666,0.22788217,0.493397350,0.167379396,0.2631579,0.3333333,0.1666667,0.2857143
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,5,12.621379,0.0038418899,0.15670738,0.949961205,0.015444871,0.3157895,0.1666667,0.3333333,0.4285714
7,1,6,5,15.554136,0.0044122488,0.10225982,0.771261662,-0.071463197,0.2631579,0.1666667,0.1666667,0.4285714
8,2,1,2,18.470610,0.0375585990,0.44823218,0.667831984,0.108645269,0.3333333,0.3333333,0.3333333,0.3333333
9,2,2,5,17.038136,0.0134352472,0.04859062,0.058336892,-0.430166160,0.2500000,0.2857143,0.3333333,0.1428571
10,2,3,2,10.101897,0.0153080841,0.63421665,0.289190824,0.256472019,0.2631579,0.3333333,0.1666667,0.2857143
11,2,4,3,11.801821,0.0115749762,0.26393444,0.291736624,0.255167346,0.6315789,0.3333333,0.8333333,0.7142857


In [70]:
table = cbind(lm.list$results, rf.list$results[,-c(1,2)], knn.list$results[,-c(1,2)])
table

,biome,cluster,lm.Npred,lm.RMSE,lm.bias,lm.RVar,lm.cor.pvalue,lm.cor,lm.acc,lm.acc.t1,...,k,knn.RMSE,knn.bias,knn.RVar,knn.cor.pvalue,knn.cor,knn.acc,knn.acc.t1,knn.acc.t2,knn.acc.t3
2,1,1,3,644.1613,0.22546322,0.6091194,0.024138453,0.5285243,0.4444444,0.5000000,...,3,14.878849,0.0035797410,0.28702830,0.006411374,-0.601783979,0.2105263,0.1666667,0.3333333,0.1428571
3,1,2,2,814.5127,0.22037978,0.5546609,0.002693328,0.6632875,0.5000000,0.5000000,...,3,18.865581,0.0075417527,0.24112970,0.224008423,0.292665424,0.3157895,0.3333333,0.1666667,0.4285714
4,1,3,1,1198.8092,0.63439353,0.3739380,0.072363808,0.4334172,0.5000000,0.5000000,...,4,25.877685,0.0162244666,0.22788217,0.493397350,0.167379396,0.2631579,0.3333333,0.1666667,0.2857143
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,2,1342.5594,0.67584986,0.4748432,0.014466709,0.5654238,0.5000000,0.6666667,...,5,12.621379,0.0038418899,0.15670738,0.949961205,0.015444871,0.3157895,0.1666667,0.3333333,0.4285714
7,1,6,1,1167.0856,0.32073172,0.2224139,0.184705134,0.3274423,0.5000000,0.5000000,...,5,15.554136,0.0044122488,0.10225982,0.771261662,-0.071463197,0.2631579,0.1666667,0.1666667,0.4285714
8,2,1,0,NA,NA,NA,NA,NA,NA,NA,...,2,18.470610,0.0375585990,0.44823218,0.667831984,0.108645269,0.3333333,0.3333333,0.3333333,0.3333333
9,2,2,0,NA,NA,NA,NA,NA,NA,NA,...,5,17.038136,0.0134352472,0.04859062,0.058336892,-0.430166160,0.2500000,0.2857143,0.3333333,0.1428571
10,2,3,2,1645.8899,6.34254428,0.6090327,0.002455416,0.6678372,0.6111111,0.6666667,...,2,10.101897,0.0153080841,0.63421665,0.289190824,0.256472019,0.2631579,0.3333333,0.1666667,0.2857143
11,2,4,0,NA,NA,NA,NA,NA,NA,NA,...,3,11.801821,0.0115749762,0.26393444,0.291736624,0.255167346,0.6315789,0.3333333,0.8333333,0.7142857


In [71]:
save(table, file = 'ModelData/models_results_per_cluster_with_delta_no_persistence_v2.Rdata')
#model.deltas.noP = list('lm' = lm.list, 'rf' = rf.list, 'knn' = knn.list)
#save(model.deltas.noP, file = 'ModelData/models_with_delta_no_persistence')

In [12]:
load('ModelData/models_results_per_cluster_with_delta_no_persistence_v2.Rdata', verbose = T)

Loading objects:
  table


In [20]:
# Clusters that have a linear, a random forest and a knn models
table1 <- table[-which(is.na(table$lm.RMSE)),]
table1

,biome,cluster,lm.Npred,lm.RMSE,lm.bias,lm.RVar,lm.cor.pvalue,lm.cor,lm.acc,lm.acc.t1,...,rf.acc.t3,knn.RMSE,knn.bias,knn.RVar,knn.cor.pvalue,knn.cor,knn.acc,knn.acc.t1,knn.acc.t2,knn.acc.t3
2,1,1,3,749.1626,7.358777e-01,0.5034432,0.082760227,0.4199134,0.3333333,0.5000000,...,0.1666667,1066.4974,1.771319e+00,1.5251974,0.259206247,0.2806964753,0.2222222,0.3333333,0.1666667,0.1666667
3,1,2,3,1060.0622,1.180476e+00,0.5529520,0.009843733,0.5907111,0.6111111,0.6666667,...,0.6666667,1839.9286,-3.291773e+00,0.9372088,0.953613072,0.0147705976,0.3888889,0.1666667,0.6666667,0.3333333
7,1,6,2,781.3551,3.598807e-01,0.6145972,0.001623450,0.6873124,0.5000000,0.5000000,...,0.5000000,1209.4764,1.077524e+00,0.8655176,0.171255776,0.3371513498,0.5000000,0.5000000,0.3333333,0.6666667
8,1,7,1,1308.4693,8.705650e-01,0.3591333,0.093828847,0.4068374,0.3333333,0.5000000,...,0.3333333,1869.3691,4.803949e+00,0.6930703,0.926096688,0.0235508811,0.2222222,0.3333333,0.0000000,0.3333333
10,1,9,2,911.2257,4.705580e-01,0.3907031,0.084254839,0.4180761,0.5000000,0.6666667,...,0.3333333,854.1620,5.162756e-01,0.9131021,0.008166070,0.6023026339,0.6666667,0.8333333,0.5000000,0.6666667
11,1,10,2,962.2227,4.821645e-01,0.6356550,0.001488861,0.6759363,0.5789474,0.6666667,...,0.4285714,1437.1692,1.836236e-01,0.9551255,0.125407094,0.3640946492,0.3684211,0.3333333,0.0000000,0.7142857
14,1,13,2,695.6622,3.579939e-01,0.6427620,0.001402165,0.6938825,0.6111111,0.6666667,...,0.5000000,822.2437,1.924735e-02,0.7231810,0.011073661,0.5831849638,0.5555556,0.5000000,0.5000000,0.6666667
18,2,3,2,1645.8899,6.342544e+00,0.6090327,0.002455416,0.6678372,0.6111111,0.6666667,...,0.5000000,2736.9058,-2.372302e+01,0.8464250,0.378130499,0.2210161304,0.3333333,0.5000000,0.1666667,0.3333333
20,2,5,1,804.4639,8.251074e-01,0.4205557,0.059473173,0.4523211,0.3888889,0.5000000,...,0.1666667,1143.7128,4.840288e-02,0.7454142,0.870785944,0.0412873208,0.2777778,0.1666667,0.1666667,0.5000000
21,2,6,1,430.8448,5.572789e-01,0.4815469,0.014586932,0.5648583,0.6111111,0.5000000,...,0.5000000,682.0690,-1.745615e-01,1.0451662,0.488139331,0.1746830137,0.4444444,0.5000000,0.5000000,0.3333333


# With 1 month persistence

In [72]:
lm.list <- lm.all(fireSeason_def_2, log1p(masked_ba_series), dates, corr.df.delta.1, 
                  list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', t = 1, useDeltas = T)
lm.list$results

,biome,cluster,lm.Npred,lm.RMSE,lm.bias,lm.RVar,lm.cor.pvalue,lm.cor,lm.acc,lm.acc.t1,lm.acc.t2,lm.acc.t3
2,1,1,1,641.3137,0.2135222,0.3440181,0.0424534937,0.4827168,0.3888889,0.6666667,0.3333333,0.1666667
3,1,2,2,752.3814,0.2000384,0.6189304,0.0007185666,0.7218769,0.6111111,0.6666667,0.5000000,0.6666667
4,1,3,1,1140.1274,0.6344844,0.4132092,0.0323345778,0.5055423,0.6111111,0.6666667,0.5000000,0.6666667
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,1,1073.5404,0.5016766,0.6374318,0.0003722483,0.7465685,0.5555556,0.5000000,0.5000000,0.6666667
7,1,6,1,1135.1981,0.3557503,0.2789868,0.1066567157,0.3930117,0.5555556,0.5000000,0.5000000,0.6666667
8,2,1,2,779.6275,1.7744491,0.5040408,0.0243908728,0.5426907,0.5294118,0.5000000,0.4000000,0.6666667
9,2,2,0,NA,NA,NA,NA,NA,NA,NA,NA,NA
10,2,3,2,1805.8626,6.0582693,0.5195197,0.0115488459,0.5804560,0.4444444,0.5000000,0.3333333,0.5000000
11,2,4,1,1071.5144,1.0494196,0.2328123,0.1528430649,0.3513191,0.4444444,0.6666667,0.3333333,0.3333333


In [73]:
rf.list <- rf.all(fireSeason_def_2, log1p(masked_ba_series), dates, corr.df.delta.1, 
                  list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', t = 1, useDeltas = T)
rf.list$results

,biome,cluster,rf.mtry,rf.Ntree,rf.maxnodes,rf.RMSE,rf.bias,rf.RVar,rf.cor.pvalue,rf.cor,rf.acc,rf.acc.t1,rf.acc.t2,rf.acc.t3
2,1,1,3,100,7,12.043180,0.0036252446,0.12755036,0.394390660,-0.2073285360,0.4210526,0.3333333,0.3333333,0.5714286
3,1,2,1,100,11,20.692037,0.0074938632,0.04438985,0.308751031,-0.2466204301,0.4210526,0.3333333,0.5000000,0.4285714
4,1,3,2,175,9,27.336532,0.0163494164,0.08808394,0.471135505,-0.1759741747,0.4210526,0.3333333,0.6666667,0.2857143
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,2,75,5,12.581010,0.0038363671,0.07846591,0.668623277,-0.1050584889,0.3157895,0.1666667,0.5000000,0.2857143
7,1,6,1,10,5,17.915885,0.0044525639,0.14802975,0.026814446,-0.5067446490,0.2105263,0.0000000,0.3333333,0.2857143
8,2,1,8,125,7,15.768969,0.0362589797,0.28730119,0.306848156,0.2551552953,0.4444444,0.5000000,0.3333333,0.5000000
9,2,2,7,75,11,18.505578,0.0132972481,0.12742591,0.043454631,-0.4557138580,0.3000000,0.2857143,0.5000000,0.1428571
10,2,3,8,10,11,9.199470,0.0151718063,0.25981591,0.320030964,0.2411094099,0.5789474,0.5000000,0.6666667,0.5714286
11,2,4,2,25,7,11.029273,0.0116727580,0.12926893,0.137577123,0.3535408726,0.5789474,0.5000000,0.5000000,0.7142857


In [74]:
knn.list <- knn.all(fireSeason_def_2, log1p(masked_ba_series), dates, corr.df.delta.1, 
                  list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', t = 1, useDeltas = T)
knn.list$results

,biome,cluster,k,knn.RMSE,knn.bias,knn.RVar,knn.cor.pvalue,knn.cor,knn.acc,knn.acc.t1,knn.acc.t2,knn.acc.t3
2,1,1,3,14.878849,0.0035797410,0.28702830,0.006411374,-0.601783979,0.2105263,0.1666667,0.3333333,0.1428571
3,1,2,3,18.865581,0.0075417527,0.24112970,0.224008423,0.292665424,0.3157895,0.3333333,0.1666667,0.4285714
4,1,3,4,25.877685,0.0162244666,0.22788217,0.493397350,0.167379396,0.2631579,0.3333333,0.1666667,0.2857143
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,5,12.621379,0.0038418899,0.15670738,0.949961205,0.015444871,0.3157895,0.1666667,0.3333333,0.4285714
7,1,6,5,15.554136,0.0044122488,0.10225982,0.771261662,-0.071463197,0.2631579,0.1666667,0.1666667,0.4285714
8,2,1,2,18.470610,0.0375585990,0.44823218,0.667831984,0.108645269,0.3333333,0.3333333,0.3333333,0.3333333
9,2,2,5,17.038136,0.0134352472,0.04859062,0.058336892,-0.430166160,0.2500000,0.2857143,0.3333333,0.1428571
10,2,3,2,10.101897,0.0153080841,0.63421665,0.289190824,0.256472019,0.2631579,0.3333333,0.1666667,0.2857143
11,2,4,3,11.801821,0.0115749762,0.26393444,0.291736624,0.255167346,0.6315789,0.3333333,0.8333333,0.7142857


In [75]:
table = cbind(lm.list$results, rf.list$results[,-c(1,2)], knn.list$results[,-c(1,2)])
table

,biome,cluster,lm.Npred,lm.RMSE,lm.bias,lm.RVar,lm.cor.pvalue,lm.cor,lm.acc,lm.acc.t1,...,k,knn.RMSE,knn.bias,knn.RVar,knn.cor.pvalue,knn.cor,knn.acc,knn.acc.t1,knn.acc.t2,knn.acc.t3
2,1,1,1,641.3137,0.2135222,0.3440181,0.0424534937,0.4827168,0.3888889,0.6666667,...,3,14.878849,0.0035797410,0.28702830,0.006411374,-0.601783979,0.2105263,0.1666667,0.3333333,0.1428571
3,1,2,2,752.3814,0.2000384,0.6189304,0.0007185666,0.7218769,0.6111111,0.6666667,...,3,18.865581,0.0075417527,0.24112970,0.224008423,0.292665424,0.3157895,0.3333333,0.1666667,0.4285714
4,1,3,1,1140.1274,0.6344844,0.4132092,0.0323345778,0.5055423,0.6111111,0.6666667,...,4,25.877685,0.0162244666,0.22788217,0.493397350,0.167379396,0.2631579,0.3333333,0.1666667,0.2857143
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,1,1073.5404,0.5016766,0.6374318,0.0003722483,0.7465685,0.5555556,0.5000000,...,5,12.621379,0.0038418899,0.15670738,0.949961205,0.015444871,0.3157895,0.1666667,0.3333333,0.4285714
7,1,6,1,1135.1981,0.3557503,0.2789868,0.1066567157,0.3930117,0.5555556,0.5000000,...,5,15.554136,0.0044122488,0.10225982,0.771261662,-0.071463197,0.2631579,0.1666667,0.1666667,0.4285714
8,2,1,2,779.6275,1.7744491,0.5040408,0.0243908728,0.5426907,0.5294118,0.5000000,...,2,18.470610,0.0375585990,0.44823218,0.667831984,0.108645269,0.3333333,0.3333333,0.3333333,0.3333333
9,2,2,0,NA,NA,NA,NA,NA,NA,NA,...,5,17.038136,0.0134352472,0.04859062,0.058336892,-0.430166160,0.2500000,0.2857143,0.3333333,0.1428571
10,2,3,2,1805.8626,6.0582693,0.5195197,0.0115488459,0.5804560,0.4444444,0.5000000,...,2,10.101897,0.0153080841,0.63421665,0.289190824,0.256472019,0.2631579,0.3333333,0.1666667,0.2857143
11,2,4,1,1071.5144,1.0494196,0.2328123,0.1528430649,0.3513191,0.4444444,0.6666667,...,3,11.801821,0.0115749762,0.26393444,0.291736624,0.255167346,0.6315789,0.3333333,0.8333333,0.7142857


In [76]:
table.deltas.Lag.1 = table
save(table.deltas.Lag.1, file = 'ModelData/models_results_per_cluster_with_delta_lag1_v2.Rdata')
#model.deltas.Lag.1 = list('lm' = lm.list, 'rf' = rf.list, 'knn' = knn.list)
#save(model.deltas.Lag.1, file = 'ModelData/models_with_delta_lag_1')

In [27]:
# Clusters that have a linear, a random forest and a knn models
table1 <- table[-which(is.na(table$lm.RMSE)),]
table1

,biome,cluster,lm.Npred,lm.RMSE,lm.bias,lm.RVar,lm.cor.pvalue,lm.cor,lm.acc,lm.acc.t1,...,rf.acc.t3,knn.RMSE,knn.bias,knn.RVar,knn.cor.pvalue,knn.cor,knn.acc,knn.acc.t1,knn.acc.t2,knn.acc.t3
2,1,1,4,791.0618,1.0524260,0.6063437,0.1188684671,0.3809154,0.2222222,0.5000000,...,0.3333333,849.9609,-0.16619529,0.9716504,0.084336693,0.417976127,0.5555556,0.8333333,0.3333333,0.5000000
3,1,2,5,1097.7271,0.8057144,0.7412926,0.0097978371,0.5910063,0.6111111,0.5000000,...,0.6666667,1590.7447,-1.96224815,0.5724783,0.772106500,0.073444756,0.2777778,0.3333333,0.1666667,0.3333333
7,1,6,4,849.2544,0.2832371,0.6581756,0.0049843906,0.6309989,0.5000000,0.5000000,...,0.5000000,1061.3890,0.61367123,0.8778955,0.045251545,0.477151301,0.6111111,0.6666667,0.5000000,0.6666667
8,1,7,2,1103.2209,0.8848320,0.5240503,0.0055722913,0.6247501,0.4444444,0.5000000,...,0.5000000,1777.7685,3.77392071,0.8620613,0.491210814,0.173470528,0.2777778,0.1666667,0.1666667,0.5000000
9,1,8,1,1838.6705,1.4914485,0.2433652,0.2332702016,0.2958470,0.4444444,0.5000000,...,0.5000000,2579.3998,2.01012252,0.9776019,0.830924490,0.054179019,0.2777778,0.1666667,0.1666667,0.5000000
10,1,9,2,911.2257,0.4705580,0.3907031,0.0842548389,0.4180761,0.5000000,0.6666667,...,0.3333333,998.4697,0.60809681,1.0569955,0.036809964,0.494854921,0.6111111,0.6666667,0.5000000,0.6666667
11,1,10,4,1087.8211,0.4330255,0.6553799,0.0089653082,0.5818408,0.4736842,0.6666667,...,0.7142857,1714.8105,1.87406979,1.1400230,0.382190026,0.212606472,0.3684211,0.3333333,0.3333333,0.4285714
14,1,13,3,721.2508,0.3330259,0.6159931,0.0025181304,0.6666038,0.6111111,0.6666667,...,0.5000000,801.3465,0.55440828,0.8369258,0.006121086,0.619383135,0.6111111,0.8333333,0.3333333,0.6666667
16,2,1,2,779.6275,1.7744491,0.5040408,0.0243908728,0.5426907,0.5294118,0.5000000,...,0.3333333,1136.6713,0.24066167,1.0932755,0.314105555,0.259706307,0.2941176,0.1666667,0.2000000,0.5000000
18,2,3,4,1715.1526,6.8565361,0.6318661,0.0042528435,0.6396746,0.5555556,0.5000000,...,0.5000000,3022.5235,-16.00587192,0.8233637,0.955959438,-0.014022760,0.2777778,0.3333333,0.1666667,0.3333333


# With 2-months persistence

In [77]:
lm.list <- lm.all(fireSeason_def_2, log1p(masked_ba_series), dates, corr.df.delta.2, 
                  list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', t = 2, useDeltas = T)
lm.list$results

,biome,cluster,lm.Npred,lm.RMSE,lm.bias,lm.RVar,lm.cor.pvalue,lm.cor,lm.acc,lm.acc.t1,lm.acc.t2,lm.acc.t3
2,1,1,1,686.1236,0.1589643,0.2605746,0.1396178017,0.3622352,0.4444444,0.3333333,0.3333333,0.6666667
3,1,2,2,887.6048,0.2014848,0.4704567,0.0113641158,0.5815061,0.5000000,0.6666667,0.5000000,0.3333333
4,1,3,2,1144.2409,0.4957427,0.5453157,0.0249853248,0.5258904,0.5555556,0.5000000,0.5000000,0.6666667
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,1,1075.6070,0.4910581,0.6197156,0.0003910237,0.7448105,0.6111111,0.6666667,0.5000000,0.6666667
7,1,6,2,1113.5677,0.3261534,0.3602208,0.0674025860,0.4403713,0.5000000,0.5000000,0.3333333,0.6666667
8,2,1,0,NA,NA,NA,NA,NA,NA,NA,NA,NA
9,2,2,0,NA,NA,NA,NA,NA,NA,NA,NA,NA
10,2,3,2,1631.6086,6.7038344,0.6315470,0.0020537681,0.6764175,0.5555556,0.5000000,0.5000000,0.6666667
11,2,4,0,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [78]:
rf.list <- rf.all(fireSeason_def_2, log1p(masked_ba_series), dates, corr.df.delta.2, 
                  list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', t = 2, useDeltas = T)
rf.list$results

,biome,cluster,rf.mtry,rf.Ntree,rf.maxnodes,rf.RMSE,rf.bias,rf.RVar,rf.cor.pvalue,rf.cor,rf.acc,rf.acc.t1,rf.acc.t2,rf.acc.t3
2,1,1,3,100,7,12.043180,0.0036252446,0.12755036,0.394390660,-0.2073285360,0.4210526,0.3333333,0.3333333,0.5714286
3,1,2,1,100,11,20.692037,0.0074938632,0.04438985,0.308751031,-0.2466204301,0.4210526,0.3333333,0.5000000,0.4285714
4,1,3,2,175,9,27.336532,0.0163494164,0.08808394,0.471135505,-0.1759741747,0.4210526,0.3333333,0.6666667,0.2857143
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,2,75,5,12.581010,0.0038363671,0.07846591,0.668623277,-0.1050584889,0.3157895,0.1666667,0.5000000,0.2857143
7,1,6,1,10,5,17.915885,0.0044525639,0.14802975,0.026814446,-0.5067446490,0.2105263,0.0000000,0.3333333,0.2857143
8,2,1,8,125,7,15.768969,0.0362589797,0.28730119,0.306848156,0.2551552953,0.4444444,0.5000000,0.3333333,0.5000000
9,2,2,7,75,11,18.505578,0.0132972481,0.12742591,0.043454631,-0.4557138580,0.3000000,0.2857143,0.5000000,0.1428571
10,2,3,8,10,11,9.199470,0.0151718063,0.25981591,0.320030964,0.2411094099,0.5789474,0.5000000,0.6666667,0.5714286
11,2,4,2,25,7,11.029273,0.0116727580,0.12926893,0.137577123,0.3535408726,0.5789474,0.5000000,0.5000000,0.7142857


In [79]:
knn.list <- knn.all(fireSeason_def_2, log1p(masked_ba_series), dates, corr.df.delta.2, 
                  list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', t = 2, useDeltas = T)
knn.list$results

,biome,cluster,k,knn.RMSE,knn.bias,knn.RVar,knn.cor.pvalue,knn.cor,knn.acc,knn.acc.t1,knn.acc.t2,knn.acc.t3
2,1,1,3,14.878849,0.0035797410,0.28702830,0.006411374,-0.601783979,0.2105263,0.1666667,0.3333333,0.1428571
3,1,2,3,18.865581,0.0075417527,0.24112970,0.224008423,0.292665424,0.3157895,0.3333333,0.1666667,0.4285714
4,1,3,4,25.877685,0.0162244666,0.22788217,0.493397350,0.167379396,0.2631579,0.3333333,0.1666667,0.2857143
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,5,12.621379,0.0038418899,0.15670738,0.949961205,0.015444871,0.3157895,0.1666667,0.3333333,0.4285714
7,1,6,5,15.554136,0.0044122488,0.10225982,0.771261662,-0.071463197,0.2631579,0.1666667,0.1666667,0.4285714
8,2,1,2,18.470610,0.0375585990,0.44823218,0.667831984,0.108645269,0.3333333,0.3333333,0.3333333,0.3333333
9,2,2,5,17.038136,0.0134352472,0.04859062,0.058336892,-0.430166160,0.2500000,0.2857143,0.3333333,0.1428571
10,2,3,2,10.101897,0.0153080841,0.63421665,0.289190824,0.256472019,0.2631579,0.3333333,0.1666667,0.2857143
11,2,4,3,11.801821,0.0115749762,0.26393444,0.291736624,0.255167346,0.6315789,0.3333333,0.8333333,0.7142857


In [80]:
table = cbind(lm.list$results, rf.list$results[,-c(1,2)], knn.list$results[,-c(1,2)])
table

,biome,cluster,lm.Npred,lm.RMSE,lm.bias,lm.RVar,lm.cor.pvalue,lm.cor,lm.acc,lm.acc.t1,...,k,knn.RMSE,knn.bias,knn.RVar,knn.cor.pvalue,knn.cor,knn.acc,knn.acc.t1,knn.acc.t2,knn.acc.t3
2,1,1,1,686.1236,0.1589643,0.2605746,0.1396178017,0.3622352,0.4444444,0.3333333,...,3,14.878849,0.0035797410,0.28702830,0.006411374,-0.601783979,0.2105263,0.1666667,0.3333333,0.1428571
3,1,2,2,887.6048,0.2014848,0.4704567,0.0113641158,0.5815061,0.5000000,0.6666667,...,3,18.865581,0.0075417527,0.24112970,0.224008423,0.292665424,0.3157895,0.3333333,0.1666667,0.4285714
4,1,3,2,1144.2409,0.4957427,0.5453157,0.0249853248,0.5258904,0.5555556,0.5000000,...,4,25.877685,0.0162244666,0.22788217,0.493397350,0.167379396,0.2631579,0.3333333,0.1666667,0.2857143
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,1,1075.6070,0.4910581,0.6197156,0.0003910237,0.7448105,0.6111111,0.6666667,...,5,12.621379,0.0038418899,0.15670738,0.949961205,0.015444871,0.3157895,0.1666667,0.3333333,0.4285714
7,1,6,2,1113.5677,0.3261534,0.3602208,0.0674025860,0.4403713,0.5000000,0.5000000,...,5,15.554136,0.0044122488,0.10225982,0.771261662,-0.071463197,0.2631579,0.1666667,0.1666667,0.4285714
8,2,1,0,NA,NA,NA,NA,NA,NA,NA,...,2,18.470610,0.0375585990,0.44823218,0.667831984,0.108645269,0.3333333,0.3333333,0.3333333,0.3333333
9,2,2,0,NA,NA,NA,NA,NA,NA,NA,...,5,17.038136,0.0134352472,0.04859062,0.058336892,-0.430166160,0.2500000,0.2857143,0.3333333,0.1428571
10,2,3,2,1631.6086,6.7038344,0.6315470,0.0020537681,0.6764175,0.5555556,0.5000000,...,2,10.101897,0.0153080841,0.63421665,0.289190824,0.256472019,0.2631579,0.3333333,0.1666667,0.2857143
11,2,4,0,NA,NA,NA,NA,NA,NA,NA,...,3,11.801821,0.0115749762,0.26393444,0.291736624,0.255167346,0.6315789,0.3333333,0.8333333,0.7142857


In [81]:
table.deltas.Lag.2 = table
save(table.deltas.Lag.2, file = 'ModelData/models_results_per_cluster_with_delta_lag2_v2.Rdata')
#model.deltas.Lag.2 = list('lm' = lm.list, 'rf' = rf.list, 'knn' = knn.list)
#save(model.deltas.Lag.2, file = 'ModelData/models_with_delta_lag_2')

In [34]:
# Clusters that have a linear, a random forest and a knn models
table1 <- table[-which(is.na(table$lm.RMSE)),]
table1

,biome,cluster,lm.Npred,lm.RMSE,lm.bias,lm.RVar,lm.cor.pvalue,lm.cor,lm.acc,lm.acc.t1,...,rf.acc.t3,knn.RMSE,knn.bias,knn.RVar,knn.cor.pvalue,knn.cor,knn.acc,knn.acc.t1,knn.acc.t2,knn.acc.t3
2,1,1,5,790.3933,1.2114526,0.6790436,0.0950080145,0.4055116,0.2777778,0.6666667,...,0.3333333,1074.6415,0.82277968,1.4017366,0.369646868,0.22487406,0.6111111,0.6666667,0.3333333,0.8333333
3,1,2,8,1385.2171,2.0053355,1.2243384,0.0398915338,0.4880637,0.5555556,0.5000000,...,0.6666667,1460.7292,-1.85896244,0.6056185,0.330926954,0.24315321,0.3888889,0.5000000,0.3333333,0.3333333
7,1,6,6,919.1487,0.1706863,0.9313706,0.0064237718,0.6165882,0.5000000,0.6666667,...,0.5000000,1116.7923,-0.07238596,1.0609969,0.046758861,0.47426091,0.6111111,0.6666667,0.5000000,0.6666667
8,1,7,4,993.6085,0.8954892,0.7004217,0.0008387456,0.7156758,0.6111111,0.6666667,...,0.3333333,1950.5799,3.78605383,0.9074070,0.934013606,0.02102246,0.1111111,0.0000000,0.0000000,0.3333333
9,1,8,2,1670.8000,1.2405577,0.4644307,0.0356661209,0.4974869,0.6111111,0.5000000,...,0.5000000,2671.0379,0.40122268,1.0950241,0.873314107,0.04047323,0.2777778,0.1666667,0.1666667,0.5000000
10,1,9,3,946.2030,0.3855469,0.5045371,0.1001410298,0.3998750,0.5000000,0.5000000,...,0.1666667,1093.6464,0.60488372,1.1847662,0.074609991,0.43038432,0.6111111,0.6666667,0.5000000,0.6666667
11,1,10,5,1021.1538,0.3260403,0.8335214,0.0021063985,0.6599661,0.6315789,0.6666667,...,0.5714286,1417.8145,1.47332665,0.9224698,0.095109215,0.39398206,0.4736842,0.6666667,0.3333333,0.4285714
12,1,11,1,634.4506,0.9263493,0.3356955,0.1850659755,0.3271884,0.5555556,0.6666667,...,0.5000000,835.6965,1.48039912,1.0893537,0.392139657,0.21474918,0.2777778,0.3333333,0.0000000,0.5000000
14,1,13,5,804.9548,0.5600556,0.7364937,0.0087477491,0.5980831,0.4444444,0.5000000,...,0.5000000,833.1974,-0.28656701,0.6136460,0.015074552,0.56260176,0.6666667,0.6666667,0.5000000,0.8333333
16,2,1,2,779.6275,1.7744491,0.5040408,0.0243908728,0.5426907,0.5294118,0.5000000,...,0.6666667,1195.0883,8.94165535,0.3335826,0.550251158,-0.15586927,0.5294118,0.5000000,0.4000000,0.6666667


# With 3-months persistence

In [82]:
lm.list <- lm.all(fireSeason_def_2, log1p(masked_ba_series), dates, corr.df.delta.3, 
                  list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', t = 3, useDeltas = T)
lm.list$results

,biome,cluster,lm.Npred,lm.RMSE,lm.bias,lm.RVar,lm.cor.pvalue,lm.cor,lm.acc,lm.acc.t1,lm.acc.t2,lm.acc.t3
2,1,1,2,583.9613,0.19888897,0.6027327,0.0064142416,0.6166746,0.5000000,0.6666667,0.3333333,0.5000000
3,1,2,3,899.0763,0.45665416,0.5928941,0.0104225401,0.5870807,0.5000000,0.6666667,0.3333333,0.5000000
4,1,3,1,949.4636,0.58760102,0.5703419,0.0015566525,0.6892131,0.7222222,0.6666667,0.6666667,0.8333333
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,3,1546.7313,0.50733873,0.5339227,0.0851030136,0.4170440,0.5000000,0.6666667,0.3333333,0.5000000
7,1,6,3,893.7830,0.17955442,0.6870041,0.0013743676,0.6947676,0.5000000,0.6666667,0.3333333,0.5000000
8,2,1,0,NA,NA,NA,NA,NA,NA,NA,NA,NA
9,2,2,0,NA,NA,NA,NA,NA,NA,NA,NA,NA
10,2,3,2,1498.7378,3.09737649,0.6691557,0.0005491549,0.7322905,0.6666667,0.8333333,0.5000000,0.6666667
11,2,4,0,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [83]:
rf.list <- rf.all(fireSeason_def_2, log1p(masked_ba_series), dates, corr.df.delta.3, 
                  list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', t = 3, useDeltas = T)
rf.list$results

,biome,cluster,rf.mtry,rf.Ntree,rf.maxnodes,rf.RMSE,rf.bias,rf.RVar,rf.cor.pvalue,rf.cor,rf.acc,rf.acc.t1,rf.acc.t2,rf.acc.t3
2,1,1,3,100,7,12.043180,0.0036252446,0.12755036,0.394390660,-0.2073285360,0.4210526,0.3333333,0.3333333,0.5714286
3,1,2,1,100,11,20.692037,0.0074938632,0.04438985,0.308751031,-0.2466204301,0.4210526,0.3333333,0.5000000,0.4285714
4,1,3,2,175,9,27.336532,0.0163494164,0.08808394,0.471135505,-0.1759741747,0.4210526,0.3333333,0.6666667,0.2857143
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,2,75,5,12.581010,0.0038363671,0.07846591,0.668623277,-0.1050584889,0.3157895,0.1666667,0.5000000,0.2857143
7,1,6,1,10,5,17.915885,0.0044525639,0.14802975,0.026814446,-0.5067446490,0.2105263,0.0000000,0.3333333,0.2857143
8,2,1,8,125,7,15.768969,0.0362589797,0.28730119,0.306848156,0.2551552953,0.4444444,0.5000000,0.3333333,0.5000000
9,2,2,7,75,11,18.505578,0.0132972481,0.12742591,0.043454631,-0.4557138580,0.3000000,0.2857143,0.5000000,0.1428571
10,2,3,8,10,11,9.199470,0.0151718063,0.25981591,0.320030964,0.2411094099,0.5789474,0.5000000,0.6666667,0.5714286
11,2,4,2,25,7,11.029273,0.0116727580,0.12926893,0.137577123,0.3535408726,0.5789474,0.5000000,0.5000000,0.7142857


In [84]:
knn.list <- knn.all(fireSeason_def_2, log1p(masked_ba_series), dates, corr.df.delta.3,
                  list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', t = 3, useDeltas = T)
knn.list$results

,biome,cluster,k,knn.RMSE,knn.bias,knn.RVar,knn.cor.pvalue,knn.cor,knn.acc,knn.acc.t1,knn.acc.t2,knn.acc.t3
2,1,1,3,14.878849,0.0035797410,0.28702830,0.006411374,-0.601783979,0.2105263,0.1666667,0.3333333,0.1428571
3,1,2,3,18.865581,0.0075417527,0.24112970,0.224008423,0.292665424,0.3157895,0.3333333,0.1666667,0.4285714
4,1,3,4,25.877685,0.0162244666,0.22788217,0.493397350,0.167379396,0.2631579,0.3333333,0.1666667,0.2857143
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,5,12.621379,0.0038418899,0.15670738,0.949961205,0.015444871,0.3157895,0.1666667,0.3333333,0.4285714
7,1,6,5,15.554136,0.0044122488,0.10225982,0.771261662,-0.071463197,0.2631579,0.1666667,0.1666667,0.4285714
8,2,1,2,18.470610,0.0375585990,0.44823218,0.667831984,0.108645269,0.3333333,0.3333333,0.3333333,0.3333333
9,2,2,5,17.038136,0.0134352472,0.04859062,0.058336892,-0.430166160,0.2500000,0.2857143,0.3333333,0.1428571
10,2,3,2,10.101897,0.0153080841,0.63421665,0.289190824,0.256472019,0.2631579,0.3333333,0.1666667,0.2857143
11,2,4,3,11.801821,0.0115749762,0.26393444,0.291736624,0.255167346,0.6315789,0.3333333,0.8333333,0.7142857


In [85]:
table = cbind(lm.list$results, rf.list$results[,-c(1,2)], knn.list$results[,-c(1,2)])
table

,biome,cluster,lm.Npred,lm.RMSE,lm.bias,lm.RVar,lm.cor.pvalue,lm.cor,lm.acc,lm.acc.t1,...,k,knn.RMSE,knn.bias,knn.RVar,knn.cor.pvalue,knn.cor,knn.acc,knn.acc.t1,knn.acc.t2,knn.acc.t3
2,1,1,2,583.9613,0.19888897,0.6027327,0.0064142416,0.6166746,0.5000000,0.6666667,...,3,14.878849,0.0035797410,0.28702830,0.006411374,-0.601783979,0.2105263,0.1666667,0.3333333,0.1428571
3,1,2,3,899.0763,0.45665416,0.5928941,0.0104225401,0.5870807,0.5000000,0.6666667,...,3,18.865581,0.0075417527,0.24112970,0.224008423,0.292665424,0.3157895,0.3333333,0.1666667,0.4285714
4,1,3,1,949.4636,0.58760102,0.5703419,0.0015566525,0.6892131,0.7222222,0.6666667,...,4,25.877685,0.0162244666,0.22788217,0.493397350,0.167379396,0.2631579,0.3333333,0.1666667,0.2857143
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,3,1546.7313,0.50733873,0.5339227,0.0851030136,0.4170440,0.5000000,0.6666667,...,5,12.621379,0.0038418899,0.15670738,0.949961205,0.015444871,0.3157895,0.1666667,0.3333333,0.4285714
7,1,6,3,893.7830,0.17955442,0.6870041,0.0013743676,0.6947676,0.5000000,0.6666667,...,5,15.554136,0.0044122488,0.10225982,0.771261662,-0.071463197,0.2631579,0.1666667,0.1666667,0.4285714
8,2,1,0,NA,NA,NA,NA,NA,NA,NA,...,2,18.470610,0.0375585990,0.44823218,0.667831984,0.108645269,0.3333333,0.3333333,0.3333333,0.3333333
9,2,2,0,NA,NA,NA,NA,NA,NA,NA,...,5,17.038136,0.0134352472,0.04859062,0.058336892,-0.430166160,0.2500000,0.2857143,0.3333333,0.1428571
10,2,3,2,1498.7378,3.09737649,0.6691557,0.0005491549,0.7322905,0.6666667,0.8333333,...,2,10.101897,0.0153080841,0.63421665,0.289190824,0.256472019,0.2631579,0.3333333,0.1666667,0.2857143
11,2,4,0,NA,NA,NA,NA,NA,NA,NA,...,3,11.801821,0.0115749762,0.26393444,0.291736624,0.255167346,0.6315789,0.3333333,0.8333333,0.7142857


In [86]:
table.deltas.Lag.3 = table
save(table.deltas.Lag.3, file = 'ModelData/models_results_per_cluster_with_delta_lag3_v2.Rdata')
#model.deltas.Lag.3 = list('lm' = lm.list, 'rf' = rf.list, 'knn' = knn.list)
#save(model.deltas.Lag.3, file = 'ModelData/models_with_delta_lag_3')

In [41]:
# Clusters that have a linear, a random forest and a knn models
table1 <- table[-which(is.na(table$lm.RMSE)),]
table1

,biome,cluster,lm.Npred,lm.RMSE,lm.bias,lm.RVar,lm.cor.pvalue,lm.cor,lm.acc,lm.acc.t1,...,rf.acc.t3,knn.RMSE,knn.bias,knn.RVar,knn.cor.pvalue,knn.cor,knn.acc,knn.acc.t1,knn.acc.t2,knn.acc.t3
2,1,1,6,702.6659,1.151362e+00,0.8677709,1.280546e-02,0.5736489,0.6111111,0.6666667,...,0.3333333,1171.1354,3.7748890,1.7987420,0.293497866,0.26205291,0.4444444,0.5000000,0.3333333,0.5000000
3,1,2,10,1739.6487,3.552391e+00,1.8420873,1.045362e-01,0.3952115,0.3333333,0.3333333,...,0.6666667,1325.9677,-2.1945618,0.9456953,0.037842522,0.49253237,0.4444444,0.5000000,0.3333333,0.5000000
4,1,3,1,727.6245,6.993964e-01,0.4628195,1.868325e-02,0.5474886,0.3888889,0.3333333,...,0.5000000,1010.2849,-0.2603100,0.6951290,0.422272239,0.20167029,0.2777778,0.5000000,0.0000000,0.3333333
7,1,6,9,878.2502,9.884007e-02,1.2335793,1.165202e-03,0.7019517,0.5555556,0.6666667,...,0.5000000,1070.7833,0.1308466,1.0604127,0.029771258,0.51218862,0.7222222,0.6666667,0.6666667,0.8333333
8,1,7,6,823.2863,6.387325e-01,0.8772220,3.351286e-05,0.8180668,0.7222222,0.8333333,...,0.5000000,1878.7452,3.2290185,1.2697197,0.358058043,0.23022532,0.2777778,0.3333333,0.1666667,0.3333333
9,1,8,3,1462.4959,1.943508e+00,0.7579878,2.553842e-03,0.6659127,0.5555556,0.5000000,...,0.6666667,2509.8351,3.5818627,1.0443479,0.565290376,0.14523370,0.4444444,0.5000000,0.3333333,0.5000000
10,1,9,4,1058.5157,4.910323e-01,0.6254316,2.479578e-01,0.2871438,0.5000000,0.5000000,...,0.3333333,1130.4630,0.2102726,1.1447929,0.121251954,0.37865846,0.5555556,0.5000000,0.5000000,0.6666667
11,1,10,8,1549.2605,-6.498365e-01,1.2941090,9.775770e-02,0.3911159,0.5789474,0.5000000,...,0.7142857,1720.0709,1.6526093,0.7934741,0.868435223,0.04075297,0.3157895,0.1666667,0.1666667,0.5714286
12,1,11,1,634.4506,9.263493e-01,0.3356955,1.850660e-01,0.3271884,0.5555556,0.6666667,...,0.1666667,1220.8132,4.4049851,1.5942810,0.508705789,-0.16663283,0.1666667,0.1666667,0.1666667,0.1666667
14,1,13,6,871.8011,4.807331e-01,0.7884631,2.133487e-02,0.5377945,0.4444444,0.5000000,...,0.3333333,939.4783,-0.2393345,0.6789134,0.062946318,0.44694927,0.5555556,0.5000000,0.5000000,0.6666667


# Combining all: no persistence and 1, 2 and 3 months persistence

In [87]:
lm.list <- lm.all(fireSeason_def_2, log1p(masked_ba_series), dates, 
                  cbind(corr.df.delta, corr.df.delta.1[,-c(1,2)], corr.df.delta.2[,-c(1,2)], corr.df.delta.3[,-c(1,2)]), 
                  list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', t = 3, useDeltas = T)
lm.list$results

,biome,cluster,lm.Npred,lm.RMSE,lm.bias,lm.RVar,lm.cor.pvalue,lm.cor,lm.acc,lm.acc.t1,lm.acc.t2,lm.acc.t3
2,1,1,7,804.3884,0.1424074,0.9456270,1.294451e-01,0.37113081,0.6111111,0.6666667,0.6666667,0.5000000
3,1,2,9,919.4942,0.7079149,1.0373864,3.292371e-03,0.65314560,0.5555556,0.5000000,0.5000000,0.6666667
4,1,3,5,1145.4001,0.8205837,0.8002434,1.234729e-02,0.57606639,0.5000000,0.5000000,0.3333333,0.6666667
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,7,1385.5811,0.4101434,0.8597187,8.099486e-03,0.60280095,0.4444444,0.6666667,0.3333333,0.3333333
7,1,6,7,757.3358,0.3946761,0.9996329,4.914801e-05,0.80836636,0.6666667,0.8333333,0.5000000,0.6666667
8,2,1,2,779.6275,1.7744491,0.5040408,2.439087e-02,0.54269066,0.5294118,0.5000000,0.4000000,0.6666667
9,2,2,0,NA,NA,NA,NA,NA,NA,NA,NA,NA
10,2,3,8,1398.8421,3.3295628,0.9263980,1.045755e-04,0.78753090,0.5555556,0.6666667,0.3333333,0.6666667
11,2,4,1,1071.5144,1.0494196,0.2328123,1.528431e-01,0.35131908,0.4444444,0.6666667,0.3333333,0.3333333


In [88]:
rf.list <- rf.all(fireSeason_def_2, log1p(masked_ba_series), dates, 
                  cbind(corr.df.delta, corr.df.delta.1[,-c(1,2)], corr.df.delta.2[,-c(1,2)], corr.df.delta.3[,-c(1,2)]), 
                  list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', t = 3, useDeltas = T)
rf.list$results

,biome,cluster,rf.mtry,rf.Ntree,rf.maxnodes,rf.RMSE,rf.bias,rf.RVar,rf.cor.pvalue,rf.cor,rf.acc,rf.acc.t1,rf.acc.t2,rf.acc.t3
2,1,1,6,10,9,11.971225,0.0036052280,0.18117908,0.716414756,-0.089227336,0.3684211,0.3333333,0.1666667,0.5714286
3,1,2,1,25,11,21.727805,0.0075334508,0.08241980,0.183036251,-0.319063247,0.1578947,0.0000000,0.3333333,0.1428571
4,1,3,1,10,11,29.379088,0.0160570089,0.12045134,0.125458056,-0.364048980,0.3157895,0.3333333,0.5000000,0.1428571
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,7,10,9,12.295767,0.0038596542,0.07533823,0.950858090,-0.015167719,0.4210526,0.5000000,0.3333333,0.4285714
7,1,6,6,150,5,17.351630,0.0044646519,0.11185265,0.030515714,-0.496698839,0.2105263,0.1666667,0.3333333,0.1428571
8,2,1,8,200,11,14.768294,0.0360604500,0.17302978,0.171580238,0.336911189,0.5000000,0.5000000,0.3333333,0.6666667
9,2,2,8,175,9,16.359082,0.0133598384,0.11338728,0.877709971,-0.036762673,0.4000000,0.5714286,0.3333333,0.2857143
10,2,3,7,175,11,8.803169,0.0151894961,0.14520316,0.240564212,0.282898701,0.3684211,0.3333333,0.3333333,0.4285714
11,2,4,8,150,7,12.047085,0.0114790794,0.09252191,0.693547583,0.096756277,0.4210526,0.3333333,0.3333333,0.5714286


In [89]:
knn.list <- knn.all(fireSeason_def_2, log1p(masked_ba_series), dates,
                    cbind(corr.df.delta, corr.df.delta.1[,-c(1,2)], corr.df.delta.2[,-c(1,2)], corr.df.delta.3[,-c(1,2)]),
                    list(nina34, nao, poleur, ea, eawr, pna, scand, wp), mode = 'unimodal', t = 3, useDeltas = T)
knn.list$results

,biome,cluster,k,knn.RMSE,knn.bias,knn.RVar,knn.cor.pvalue,knn.cor,knn.acc,knn.acc.t1,knn.acc.t2,knn.acc.t3
2,1,1,3,14.592279,0.0036282475,0.2873341,0.01555249,-0.546159804,0.1052632,0.0000000,0.1666667,0.1428571
3,1,2,2,17.713156,0.0074212482,0.6897204,0.02487869,0.512438378,0.4210526,0.6666667,0.1666667,0.4285714
4,1,3,3,27.311662,0.0163262658,0.2776248,0.73622393,0.082766728,0.2631579,0.3333333,0.1666667,0.2857143
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,4,13.014139,0.0038629352,0.1479782,0.72882304,-0.085174149,0.2105263,0.0000000,0.1666667,0.4285714
7,1,6,1,20.176518,0.0043308675,0.8286459,0.88062161,-0.036950916,0.3684211,0.3333333,0.6666667,0.1428571
8,2,1,1,18.130236,0.0376313513,0.7167432,0.26123738,0.279551088,0.3888889,0.5000000,0.3333333,0.3333333
9,2,2,2,14.446185,0.0134974287,0.1789705,0.14151979,0.340745149,0.6000000,0.8571429,0.5000000,0.4285714
10,2,3,1,9.210512,0.0151515167,0.6936852,0.08311690,0.407741913,0.4736842,0.3333333,0.1666667,0.8571429
11,2,4,2,11.466013,0.0117067789,0.3045276,0.17166464,0.327081628,0.4210526,0.3333333,0.5000000,0.4285714


In [90]:
table = cbind(lm.list$results, rf.list$results[,-c(1,2)], knn.list$results[,-c(1,2)])
table

,biome,cluster,lm.Npred,lm.RMSE,lm.bias,lm.RVar,lm.cor.pvalue,lm.cor,lm.acc,lm.acc.t1,...,k,knn.RMSE,knn.bias,knn.RVar,knn.cor.pvalue,knn.cor,knn.acc,knn.acc.t1,knn.acc.t2,knn.acc.t3
2,1,1,7,804.3884,0.1424074,0.9456270,1.294451e-01,0.37113081,0.6111111,0.6666667,...,3,14.592279,0.0036282475,0.2873341,0.01555249,-0.546159804,0.1052632,0.0000000,0.1666667,0.1428571
3,1,2,9,919.4942,0.7079149,1.0373864,3.292371e-03,0.65314560,0.5555556,0.5000000,...,2,17.713156,0.0074212482,0.6897204,0.02487869,0.512438378,0.4210526,0.6666667,0.1666667,0.4285714
4,1,3,5,1145.4001,0.8205837,0.8002434,1.234729e-02,0.57606639,0.5000000,0.5000000,...,3,27.311662,0.0163262658,0.2776248,0.73622393,0.082766728,0.2631579,0.3333333,0.1666667,0.2857143
5,1,4,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,5,7,1385.5811,0.4101434,0.8597187,8.099486e-03,0.60280095,0.4444444,0.6666667,...,4,13.014139,0.0038629352,0.1479782,0.72882304,-0.085174149,0.2105263,0.0000000,0.1666667,0.4285714
7,1,6,7,757.3358,0.3946761,0.9996329,4.914801e-05,0.80836636,0.6666667,0.8333333,...,1,20.176518,0.0043308675,0.8286459,0.88062161,-0.036950916,0.3684211,0.3333333,0.6666667,0.1428571
8,2,1,2,779.6275,1.7744491,0.5040408,2.439087e-02,0.54269066,0.5294118,0.5000000,...,1,18.130236,0.0376313513,0.7167432,0.26123738,0.279551088,0.3888889,0.5000000,0.3333333,0.3333333
9,2,2,0,NA,NA,NA,NA,NA,NA,NA,...,2,14.446185,0.0134974287,0.1789705,0.14151979,0.340745149,0.6000000,0.8571429,0.5000000,0.4285714
10,2,3,8,1398.8421,3.3295628,0.9263980,1.045755e-04,0.78753090,0.5555556,0.6666667,...,1,9.210512,0.0151515167,0.6936852,0.08311690,0.407741913,0.4736842,0.3333333,0.1666667,0.8571429
11,2,4,1,1071.5144,1.0494196,0.2328123,1.528431e-01,0.35131908,0.4444444,0.6666667,...,2,11.466013,0.0117067789,0.3045276,0.17166464,0.327081628,0.4210526,0.3333333,0.5000000,0.4285714


In [91]:
table.deltas.Lag.0123 = table
save(table.deltas.Lag.0123, file = 'ModelData/models_results_per_cluster_with_delta_lag0123_v2.Rdata')
#model.deltas.Lag.3 = list('lm' = lm.list, 'rf' = rf.list, 'knn' = knn.list)
#save(model.deltas.Lag.3, file = 'ModelData/models_with_delta_lag_3')